In [1]:
!git clone https://github.com/facebookresearch/VGGSfM.git
%cd VGGSfM


Cloning into 'VGGSfM'...
remote: Enumerating objects: 1282, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 1282 (delta 453), reused 370 (delta 369), pack-reused 764 (from 2)
Receiving objects: 100% (1282/1282), 75.02 MiB | 15.44 MiB/s, done.
Resolving deltas: 100% (759/759), done.
/content/VGGSfM


In [2]:
# Install system dependencies
!apt-get update
!apt-get install -y colmap ffmpeg

# Install Python dependencies
!pip install -U pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python hydra-core omegaconf visdom pycolmap kornia einops
!pip install git+https://github.com/cvg/LightGlue.git
!pip install git+https://github.com/facebookresearch/dinov2.git
!pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py3.10_cu118_pyt1110/download.html


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,715 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd

In [5]:
# Step 1: Install exact working versions for torch and xformers
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install xformers==0.0.29

# Step 2: Clone dinov2 and PATCH xformers requirement
!rm -rf dinov2
!git clone https://github.com/facebookresearch/dinov2.git
%cd dinov2

# Patch setup.py to remove strict xformers==0.0.18
!sed -i 's/xformers==0.0.18/xformers>=0.0.18/' setup.py

# Now install dinov2 without errors
!pip install .


Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached torch-2.5.1-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached triton-3.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
Using cached torch-2.5.1-cp311-cp311-manylinux1_x86_64.whl (906.5 MB)
Using cached triton-3.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209.5 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torch]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.

In [6]:
# Go into dinov2 folder
%cd /content/VGGSfM/dinov2

# Patch setup.py to allow newer xformers version
!sed -i 's/xformers==0.0.18/xformers>=0.0.18/' setup.py

# Now install dinov2 without the strict version error
!pip install .


/content/VGGSfM/dinov2
Processing /content/VGGSfM/dinov2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torch-2.0.0-cp311-cp311-manylinux1_x86_64.whl.metadata (24 kB)
  Using cached torchvision-0.15.0-cp311-cp311-manylinux1_x86_64.whl.metadata (11 kB)
  Using cached torchmetrics-0.10.3-py3-none-any.whl.metadata (15 kB)
  Using cached fvcore-0.1.5.post20221221.tar.gz (50 kB)
  Preparing metadata (setup.py) ... done
  Using cached iopath-0.1.10-py3-none-any.whl
INFO: pip is looking at multiple versions of dinov2 to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement xformers==0.0.18 (from dinov2) (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.16rc424, 0.0.16rc425, 0.0.16, 0.0.20, 0.0.21, 0.0.22, 0.0.22.post7, 0.0.23, 0.0.2

In [7]:
%cd /content/VGGSfM

# Optional but useful if not already linked
!ln -s dinov2 dinov2_local


/content/VGGSfM


In [12]:
!git clone https://github.com/cvg/pyceres.git


Cloning into 'pyceres'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 390 (delta 104), reused 60 (delta 55), pack-reused 240 (from 2)
Receiving objects: 100% (390/390), 146.91 KiB | 16.32 MiB/s, done.
Resolving deltas: 100% (215/215), done.


In [15]:
!apt-get install -y libabsl-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libabsl20210324
The following NEW packages will be installed:
  libabsl-dev libabsl20210324
0 upgraded, 2 newly installed, 0 to remove and 38 not upgraded.
Need to get 1,304 kB of archives.
After this operation, 7,320 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libabsl20210324 amd64 0~20210324.2-2 [387 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libabsl-dev amd64 0~20210324.2-2 [918 kB]
Fetched 1,304 kB in 2s (697 kB/s)
Selecting previously unselected package libabsl20210324:amd64.
(Reading database ... 127463 files and directories currently installed.)
Preparing to unpack .../libabsl20210324_0~20210324.2-2_amd64.deb ...
Unpacking libabsl20210324:amd64 (0~20210324.2-2) ...
Selecting previously unselected package libabsl-dev:amd64.
Preparing to unpack .../libabsl-dev_0

In [17]:
!apt-get install -y libeigen3-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  libeigen3-doc libmpfrc++-dev
The following NEW packages will be installed:
  libeigen3-dev
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 1,056 kB of archives.
After this operation, 9,081 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libeigen3-dev all 3.4.0-2ubuntu2 [1,056 kB]
Fetched 1,056 kB in 3s (389 kB/s)
Selecting previously unselected package libeigen3-dev.
(Reading database ... 128173 files and directories currently installed.)
Preparing to unpack .../libeigen3-dev_3.4.0-2ubuntu2_all.deb ...
Unpacking libeigen3-dev (3.4.0-2ubuntu2) ...
Setting up libeigen3-dev (3.4.0-2ubuntu2) ...


In [18]:
%cd /content/pyceres/ceres-solver/build
!cmake .. -DBUILD_TESTING=OFF
!make -j8
!make install


/content/pyceres/ceres-solver/build
-- Detected Ceres version: 2.3.0 from /content/pyceres/ceres-solver/include/ceres/version.h
-- ceres-solver/third_party/abseil-cpp is empty, so falling back to system installed abseil
-- Found abseil version 20210324: /usr/lib/x86_64-linux-gnu/cmake/absl
-- Found Eigen version 3.4.0: /usr/share/eigen3/cmake
-- Enabling use of Eigen as a sparse linear algebra library.
-- Found CUDA version 12.5.82 installed in: /usr/local/cuda
-- The CUDA compiler identification is NVIDIA 12.5.82 with host compiler GNU 11.4.0
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Setting CUDA Architecture to 50;60;70;75;80;90
-- Could NOT find cudss (missing: cudss_DIR)
-- Did not find cuDSS, SPARSE_SCHUR and SPARSE_NORMAL_CHOLESKY with CUDA_SPARSE will not be available.
-- Found LAPACK library: 

In [19]:
%cd /content
!git clone https://github.com/abseil/abseil-cpp.git
%cd abseil-cpp
!cmake -B build -DCMAKE_BUILD_TYPE=Release -DCMAKE_POSITION_INDEPENDENT_CODE=ON .
!cmake --build build -j8
!cmake --install build


/content
Cloning into 'abseil-cpp'...
remote: Enumerating objects: 31164, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 31164 (delta 22), reused 22 (delta 16), pack-reused 31088 (from 3)
Receiving objects: 100% (31164/31164), 17.97 MiB | 15.52 MiB/s, done.
Resolving deltas: 100% (24322/24322), done.
/content/abseil-cpp
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test ABSL_INTERNAL_AT_LEAST_CXX17
-- Performing Test ABSL_INTERNAL_AT_LEAST_CXX17 - Success
-- Performing Test ABSL_INTERNAL_AT_LEAST_CXX20
-- Performing Test ABSL_INTERNAL_AT_LEAST_CXX20 - Failed
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
CMake Warning at CMak

In [20]:
%cd /content/pyceres/ceres-solver/build
!cmake .. -DBUILD_TESTING=OFF
!make -j8
!make install


/content/pyceres/ceres-solver/build
-- Detected Ceres version: 2.3.0 from /content/pyceres/ceres-solver/include/ceres/version.h
-- ceres-solver/third_party/abseil-cpp is empty, so falling back to system installed abseil
-- Found abseil version 20210324: /usr/lib/x86_64-linux-gnu/cmake/absl
-- Found Eigen version 3.4.0: /usr/share/eigen3/cmake
-- Enabling use of Eigen as a sparse linear algebra library.
-- Found CUDA version 12.5.82 installed in: /usr/local/cuda
-- Setting CUDA Architecture to 50;60;70;75;80;90
-- Could NOT find cudss (missing: cudss_DIR)
-- Did not find cuDSS, SPARSE_SCHUR and SPARSE_NORMAL_CHOLESKY with CUDA_SPARSE will not be available.
-- Found LAPACK library: /usr/lib/x86_64-linux-gnu/libmkl_intel_lp64.so;/usr/lib/x86_64-linux-gnu/libmkl_intel_thread.so;/usr/lib/x86_64-linux-gnu/libmkl_core.so;/usr/local/lib/libiomp5.so;-lm;-ldl;-lm;-ldl
-- Found CHOLMOD headers in: /usr/include/suitesparse
-- Found CHOLMOD library: /usr/lib/x86_64-linux-gnu/libcholmod.so
-- Found 

In [21]:
# Step 1: Install latest Abseil (with logging)
%cd /content
!git clone --depth=1 https://github.com/abseil/abseil-cpp.git
%cd abseil-cpp
!cmake -B build -DCMAKE_BUILD_TYPE=Release -DCMAKE_POSITION_INDEPENDENT_CODE=ON
!cmake --build build -j8
!cmake --install build


/content
fatal: destination path 'abseil-cpp' already exists and is not an empty directory.
/content/abseil-cpp
CMake Warning at CMakeLists.txt:177 (message):
    The default and system-level install directories are unsupported except in LTS   releases of Abseil.  Please set CMAKE_INSTALL_PREFIX to install Abseil in your   source or build tree directly.    


-- Configuring done (0.2s)
-- Generating done (0.3s)
-- Build files have been written to: /content/abseil-cpp/build
[  0%] Built target spinlock_wait
[  1%] Built target strerror
[  2%] Built target log_severity
[  3%] Built target utf8_for_code_point
[  4%] Built target exponential_biased
[  5%] Built target civil_time
[  9%] Built target time_zone
[  9%] Built target log_internal_nullguard
[  9%] Built target flags_commandlineflag_internal
[ 10%] Built target leak_check
[ 11%] Built target periodic_sampler
[ 12%] Built target random_internal_platform
[ 14%] Built target raw_logging_internal
[ 14%] Built target random_internal_ra

In [22]:
# Step 2: Rebuild Ceres without tests and examples
%cd /content/pyceres/ceres-solver
!rm -rf build
!mkdir build && cd build
!cmake .. -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF -DCMAKE_PREFIX_PATH="/usr/local"
!make -j8
!make install


/content/pyceres/ceres-solver
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Error at CMakeLists.txt:11 (find_package):
  By not providing "FindCeres.cmake" in CMAKE_MODULE_PATH this project has
  asked CMake to find a package configuration file provided by "Ceres", but
  CMake did not find one.

  Could not find a package configuration file provided by "Ceres" (requested
  version 2.1) with any of the following names:

    CeresConfig.cmake
    ceres-config.cmake

  Add the installation prefix of "Ceres" to CMAKE_PREFIX_PATH or s

In [23]:
ls /usr/local/lib/cmake/Ceres


ls: cannot access '/usr/local/lib/cmake/Ceres': No such file or directory


In [24]:
# Start fresh
%cd /content
!rm -rf pyceres && mkdir pyceres
%cd pyceres

# Clone Ceres 2.1.0 (stable, no absl::log requirement)
!git clone -b 2.1.0 https://github.com/ceres-solver/ceres-solver.git

# Create build directory
%cd ceres-solver
!mkdir build && cd build

# Configure and build
%cd build
!cmake .. -DBUILD_TESTING=OFF
!make -j8
!make install


/content
/content/pyceres
Cloning into 'ceres-solver'...
remote: Enumerating objects: 23042, done.
remote: Counting objects: 100% (493/493), done.
remote: Compressing objects: 100% (313/313), done.
remote: Total 23042 (delta 287), reused 180 (delta 180), pack-reused 22549 (from 3)
Receiving objects: 100% (23042/23042), 17.90 MiB | 16.30 MiB/s, done.
Resolving deltas: 100% (16047/16047), done.
Note: switching to 'f68321e7de8929fbcdb95dd42877531e64f72f66'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

/content/pyceres/ceres-s

In [25]:
ls /usr/local/lib/cmake/Ceres


CeresConfig.cmake         CeresTargets-release.cmake  FindSuiteSparse.cmake
CeresConfigVersion.cmake  FindCXSparse.cmake
CeresTargets.cmake        FindMETIS.cmake


In [26]:
!mkdir -p /content/ceres_test_project


In [28]:
with open("/content/ceres_test_project/CMakeLists.txt", "w") as f:
    f.write("""\
cmake_minimum_required(VERSION 3.10)
project(CeresExample)

# Find the installed Ceres Solver
find_package(Ceres REQUIRED PATHS /usr/local/lib/cmake/Ceres)

# Create an executable from main.cpp
add_executable(main main.cpp)

# Link the executable with Ceres
target_link_libraries(main PRIVATE ceres)
""")


In [29]:
with open("/content/ceres_test_project/main.cpp", "w") as f:
    f.write("""\
#include "ceres/ceres.h"
#include "glog/logging.h"

// A simple cost functor for curve fitting (y = x^2)
struct CostFunctor {
  template <typename T>
  bool operator()(const T* const x, T* residual) const {
    residual[0] = T(10.0) - x[0] * x[0];  // residual = 10 - x^2
    return true;
  }
};

int main(int argc, char** argv) {
  google::InitGoogleLogging(argv[0]);

  double initial_x = 5.0;
  double x = initial_x;

  // Set up the optimization problem
  ceres::Problem problem;
  problem.AddResidualBlock(
      new ceres::AutoDiffCostFunction<CostFunctor, 1, 1>(new CostFunctor),
      nullptr, &x);

  // Configure solver options
  ceres::Solver::Options options;
  options.linear_solver_type = ceres::DENSE_QR;
  options.minimizer_progress_to_stdout = true;

  // Run the solver
  ceres::Solver::Summary summary;
  ceres::Solve(options, &problem, &summary);

  std::cout << summary.BriefReport() << "\\n";
  std::cout << "Initial x: " << initial_x << " -> Optimized x: " << x << "\\n";

  return 0;
}
""")


In [30]:
# Step into the test project directory
%cd /content/ceres_test_project

# Make a build directory
!mkdir -p build
%cd build

# Run CMake to configure the project
!cmake ..

# Build the project using make
!make


/content/ceres_test_project
/content/ceres_test_project/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Found CXSparse: /usr/include/suitesparse (found suitable version "3.2.0", minimum required is "3.2.0")
-- Found AMD headers in: /usr/include/suitesparse
-- Found AMD library: /usr/lib/x86_64-linux-gnu/libamd.so
-- Found CAMD headers in: /usr/include/suitesparse
-- Found CAMD library: /usr/lib/x86_64-linux-gnu/

In [31]:
!./main


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.125000e+02    0.00e+00    1.50e+02   0.00e+00   0.00e+00  1.00e+04        0    3.79e-05    1.44e-04
   1  2.533613e+00    1.10e+02    1.58e+01   1.50e+00   9.77e-01  3.00e+04        1    7.10e-05    3.99e-04
   2  5.353232e-03    2.53e+00    6.58e-01   3.22e-01   9.98e-01  9.00e+04        1    1.19e-05    4.24e-04
   3  3.539508e-08    5.35e-03    1.68e-03   1.63e-02   1.00e+00  2.70e+05        1    9.06e-06    4.57e-04
   4  3.795330e-18    3.54e-08    1.74e-08   4.21e-05   1.00e+00  8.10e+05        1    8.11e-06    4.77e-04
Ceres Solver Report: Iterations: 5, Initial cost: 1.125000e+02, Final cost: 3.795330e-18, Termination: CONVERGENCE
Initial x: 5 -> Optimized x: 3.16228


In [34]:
!ls /content/VGGSfM


assets		    CONTRIBUTING.md  examples	    README.md
cfgs		    demo.py	     install.sh     setup.py
CHANGELOG.txt	    dinov2	     LICENSE.txt    vggsfm
CODE_OF_CONDUCT.md  dinov2_local     minipytorch3d  video_demo.py


In [35]:
!chmod +x /content/VGGSfM/install.sh
!bash /content/VGGSfM/install.sh


/content/VGGSfM/install.sh: line 9: conda: command not found
/content/VGGSfM/install.sh: line 18: conda: command not found
/content/VGGSfM/install.sh: line 19: conda: command not found
/content/VGGSfM/install.sh: line 22: conda: command not found
/content/VGGSfM/install.sh: line 23: conda: command not found
/content/VGGSfM/install.sh: line 24: conda: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.3/709.3 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 149.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [gradio]
Cloning into 'dependency/LightGlue'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 202 (delta 97), reused 39 (delta 39), pack-reused 75 (from 2)
Receiving objects: 100% (202/202), 17.23 MiB | 17.73 MiB/s, done.
Resolving de

In [36]:
!ls /content/VGGSfM/examples


british_museum	cake  kitchen  statue


In [38]:
# Uninstall current torchvision
!pip uninstall -y torchvision

# Reinstall compatible version for PyTorch 2.0.1 with CUDA 11.7
!pip install torchvision==0.15.2+cu117 -f https://download.pytorch.org/whl/torch_stable.html


Found existing installation: torchvision 0.15.2+cu118
Uninstalling torchvision-0.15.2+cu118:
  Successfully uninstalled torchvision-0.15.2+cu118
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 82.9 MB/s eta 0:00:00


In [39]:
%cd /content/VGGSfM


/content/VGGSfM


In [40]:
!pip install -e dependency/LightGlue


Obtaining file:///content/VGGSfM/dependency/LightGlue
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lightglue (pyproject.toml) ... done
  Created wheel for lightglue: filename=lightglue-0.0-0.editable-py3-none-any.whl size=14728 sha256=06c13a7e707067980acb77199f97fed85664f30e39fa3201bb79989c99b033af
  Stored in directory: /tmp/pip-ephem-wheel-cache-55xayr8m/wheels/ec/f9/a4/9ae2ff124a6ece5f872c55002cf57b71de133a37af5d3ad890
Successfully built lightglue
  Attempting uninstall: lightglue
    Found existing installation: lightglue 0.0
    Uninstalling lightglue-0.0:
      Successfully uninstalled lightglue-0.0


In [44]:
!find cfgs -name "*.yaml"


cfgs/demo.yaml
cfgs/video_demo.yaml


In [47]:
!python demo.py --config-path cfgs --config-name demo


Poselib is available
Poselib is available
PyTorch3d is not available. Please disable visdom.
/content/VGGSfM/demo.py:17: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="cfgs/", config_name="demo")
/usr/local/lib/python3.11/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
Model Config: model_name: vggsfm_v2_0_0
seed: 0
img_size: 1024
debug: false
center_order: false
mixed_precision: fp16
extract_color: true
filter_invalid_frame: true
comple_nonvis: true
query_frame_num: 1
robust_refine: 2
BA_iters: 1
fine_tracking: false
camera_type: SIMPLE_PINHOLE
load_gt: false
viz_visualize: false
gr_visualize: false
fmat_thres: 4.0
max_reproj_error: 4.0
init_max